# Overview

Note: Learning note of Daniel warfield article [BERT — Intuitively and Exhaustively Explained](https://medium.com/towards-data-science/bert-intuitively-and-exhaustively-explained-48a24ecc1c8a)

Some code were wisted by the author.

In [1]:
!pip install -U -q transformers==4.39.3
!pip install -U -q datasets==2.18.0
!pip install -U -q nltk==3.9.1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.2 requires cubinlinker, which is not installed.
cudf 24.8.2 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.2 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.2 requires cupy-cuda11x>=12.0.0, which is not installed.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
cudf 24.8.2 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.6.0 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires py

# Load dataset

In [2]:
from datasets import load_dataset

dataset = load_dataset("wikipedia", "20220301.en", trust_remote_code=True, streaming=True)

# Extract sentences from articles

In [3]:
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [4]:
! unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [5]:
import itertools

num_articles=10

dataset_iter = iter(dataset['train'])

# getting n articles
articles=list(itertools.islice(dataset_iter, num_articles))

# getting paragraphs
paragraphs=[]
for article in articles:
    paragraphs.extend(article['text'].splitlines())

# filtering paragraphs so they're hopefully actually paragraphs
paragraps=[p for p in paragraphs if len(p)>50]

#dividing paragraphs into sentences
divided_paragraphs = []
for p in paragraphs:
    divided_paragraphs.append(nltk.sent_tokenize(p))

#only using paragraphs with 3 or more sentences
divided_paragraphs = [pls for pls in divided_paragraphs if len(pls)>=3]
len(divided_paragraphs)

498

# Constructing positive pairs and nagetive pairs

In [6]:
import random

positive_pairs = []
negative_pairs = []

num_paragraphs = len(divided_paragraphs)

for i, paragraph in enumerate(divided_paragraphs):
    for j in range(len(paragraph)-1):
        positive_pairs.append((paragraph[j], paragraph[j+1]))
        rand_par = i
        while rand_par == i:
            rand_par = random.randint(0, num_paragraphs-1)
        rand_sent = random.randint(0, len(divided_paragraphs[rand_par])-1)
        negative_pairs.append((paragraph[j], divided_paragraphs[rand_par][rand_sent]))
        
len(positive_pairs)

1873

# Setting up tokenization

In [7]:
from transformers.models.bert.tokenization_bert_fast import BertTokenizerFast

tokenizer=BertTokenizerFast.from_pretrained("google-bert/bert-base-uncased")
tokenizer

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

BertTokenizerFast(name_or_path='google-bert/bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

## Testing tokenizer

In [8]:
sentence="Here's a weird word: Withoutadoubricus."
print(f'original sentence: "{sentence}"')
demo_tokens=tokenizer([sentence])
print(f"token IDs: {demo_tokens['input_ids']}")
tokens=tokenizer.convert_ids_to_tokens(demo_tokens['input_ids'][0])
print(f'token values: {tokens}')

original sentence: "Here's a weird word: Withoutadoubricus."
token IDs: [[101, 2182, 1005, 1055, 1037, 6881, 2773, 1024, 2302, 9365, 12083, 7277, 2271, 1012, 102]]
token values: ['[CLS]', 'here', "'", 's', 'a', 'weird', 'word', ':', 'without', '##ado', '##ub', '##ric', '##us', '.', '[SEP]']


In [9]:
print(tokenizer(["Here's a weird word: Withoutadoubticus"]))

{'input_ids': [[101, 2182, 1005, 1055, 1037, 6881, 2773, 1024, 2302, 9365, 12083, 29587, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [10]:
decoded_string=tokenizer.decode(demo_tokens['input_ids'][0], skip_special_tokens=False)
decoded_string

"[CLS] here's a weird word : withoutadoubricus. [SEP]"

In [11]:
tokenizer.vocab_size

30522

In [12]:
tokenizer.added_tokens_decoder.keys()

dict_keys([0, 100, 101, 102, 103])

# Create batch

It's different from original paper, but similar.

In [13]:
from random import shuffle

shuffle(positive_pairs)
shuffle(negative_pairs)

In [14]:
"""Parallelized
"""

from tqdm import tqdm
import torch
from multiprocessing import Pool, cpu_count

#defining the device the data ends up living on
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#number of examples in the batch
batch_size = 128  # should be divisible by 2
#sequence length of model
max_input_length = 64

#defining parallelizable function for processign batches
def process_batch(batch_index):
    #establishing bounds of the batch
    start_index = batch_index * batch_size
    end_index = start_index + batch_size

    if end_index > len(positive_pairs):
        return None, None, None

    #getting the sentence pairs of the batch, and if they're pos or neg
    sentence_pairs = []
    is_positives = []

    # Creating positive pairs
    sentence_pairs.extend(positive_pairs[start_index:start_index + int(batch_size / 2)])
    is_positives.extend([1] * int(batch_size / 2))

    # Creating negative pairs
    sentence_pairs.extend(negative_pairs[start_index + int(batch_size / 2):end_index])
    is_positives.extend([0] * int(batch_size / 2))

    # Defining outputs
    # At the end of the day we need to know three things:
    #   - the tokens for the sequences in a batch
    #   - which sentence the tokens belong to, for positional encoding
    #   - if the examples in the batch are positive or negative
    # these keep track of the first two
    batch_sentence_location_tokens = []
    batch_sequence_tokens = []

    # Tokenizing pairs
    for sentence_pair in sentence_pairs:
        sentence1 = sentence_pair[0]
        sentence2 = sentence_pair[1]

        # Tokenizing both sentences
        tokens = tokenizer([sentence1, sentence2])
        sentence1_tokens = tokens['input_ids'][0]
        sentence2_tokens = tokens['input_ids'][1]

        # Trimming down tokens
        if len(sentence1_tokens) + len(sentence2_tokens) > max_input_length:
            sentence1_tokens = [101] + sentence1_tokens[-int(max_input_length / 2) + 1:]
            sentence2_tokens = sentence2_tokens[:int(max_input_length / 2) - 1] + [102]

        # Creating sentence tokens
        sentence_tokens = [0] * len(sentence1_tokens) + [1] * len(sentence2_tokens)

        # Combining and padding
        pad_num = max_input_length - (len(sentence1_tokens) + len(sentence2_tokens))
        sequence_tokens = sentence1_tokens + sentence2_tokens + [0] * pad_num
        sentence_location_tokens = sentence_tokens + [1] * pad_num

        # Adding to batch
        batch_sequence_tokens.append(sequence_tokens)
        batch_sentence_location_tokens.append(sentence_location_tokens)

    return torch.tensor(batch_sentence_location_tokens), torch.tensor(batch_sequence_tokens), torch.tensor(is_positives)

# Determine the number of batches
num_batches = len(positive_pairs) // batch_size

# Use a Pool of workers equal to the number of CPU cores
with Pool(processes=cpu_count()) as pool:
    results = list(tqdm(pool.imap(process_batch, range(num_batches)), total=num_batches))

# Filter out None results from the process_batch function
results = [result for result in results if result[0] is not None]

# Unpack results into batches
sentence_location_batches, sequence_tokens_batches, is_positives_batches = zip(*results)

# Stack tensors into final batches
sentence_location_batches = torch.stack(sentence_location_batches).to(device)
sequence_tokens_batches = torch.stack(sequence_tokens_batches).to(device)
is_positives_batches = torch.stack(is_positives_batches).to(device)

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling pa

In [15]:
sentence_location_batches.shape

torch.Size([14, 128, 64])

# Setting up masking

In [16]:
#listing out vocab for random token masking
vocab = tokenizer.get_vocab()
valid_token_ids = list(vocab.values())

def mask_batch(batch_tokens, clone=True):
    if clone:
        batch_tokens = torch.clone(batch_tokens)

    # Define the percentage of tokens to potentially mask
    replace_percentage = 0.15

    # Define tokens that should not be replaced
    excluded_tokens = {0, 100, 101, 102, 103}

    # Create a mask to identify tokens that are eligible for replacement
    eligible_mask = ~torch.isin(batch_tokens, torch.tensor(list(excluded_tokens)).to(device))

    # Count the number of eligible tokens
    num_eligible_tokens = eligible_mask.sum().item()

    # Calculate the number of tokens to potentially mask
    num_tokens_to_mask = int(num_eligible_tokens * replace_percentage)

    # Create a random permutation of eligible token indices
    eligible_indices = eligible_mask.nonzero(as_tuple=True)
    random_indices = torch.randperm(num_eligible_tokens)[:num_tokens_to_mask]

    # Create a probability distribution for replacement
    replacement_probs = torch.tensor([0.8, 0.1, 0.1])  # Probabilities for [103, random token, leave unchanged]
    replacement_choices = torch.multinomial(replacement_probs, num_tokens_to_mask, replacement=True)

    # Vector to store if a token was masked (0: not masked, 1: masked)
    masked_indicator = torch.zeros_like(batch_tokens, dtype=torch.int32)

    # Apply replacements based on sampled choices
    for i, idx in enumerate(random_indices):
        row = eligible_indices[0][idx]
        col = eligible_indices[1][idx]

        if replacement_choices[i] == 0:
            batch_tokens[row, col] = 103
            masked_indicator[row, col] = 1
        elif replacement_choices[i] == 1:
            batch_tokens[row, col] = random.choice(valid_token_ids)
            masked_indicator[row, col] = 1
        elif replacement_choices[i] == 2:
            masked_indicator[row, col] = 1

    return batch_tokens, masked_indicator

batch_tokens, masked_indicator = mask_batch(sequence_tokens_batches[0])
batch_tokens

tensor([[  101,   103,  3266,  ...,     0,     0,     0],
        [  101,  2715,  9201,  ...,   102,     0,     0],
        [  101, 22330,  8713,  ...,     0,     0,     0],
        ...,
        [  101,   103,  2896,  ...,     0,     0,     0],
        [  101,   103,  4472,  ...,     0,     0,     0],
        [  101,  1999,  1037,  ...,     0,     0,     0]], device='cuda:0')

# Setting up embedding

In [17]:
import torch.nn as nn
import torch

vocab_size=tokenizer.vocab_size
d_model=256
n_segments=2

class Embedding(nn.Module):
    def __init__(self):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
        self.pos_embed = nn.Embedding(max_input_length, d_model)  # position embedding
        self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, seg):
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long).to(device)
        pos = pos.unsqueeze(0).expand_as(x)  # (seq_len,) -> (batch_size, seq_len)
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        return self.norm(embedding)

embedding = Embedding()
embedding.to(device)

Embedding(
  (tok_embed): Embedding(30522, 256)
  (pos_embed): Embedding(64, 256)
  (seg_embed): Embedding(2, 256)
  (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
)

In [18]:
#in order for the input to work all sequences need to be padded to be equivilent length
dummy_embedding = embedding(sequence_tokens_batches[0], sentence_location_batches[0])
print(dummy_embedding.shape)

torch.Size([128, 64, 256])


# Defining model

The structure below:

* embedding
* encoder block
  * multi headed self-attention
  * feed forward
* a fully connected layer for outputting the prediction
* a fully connected layer for turning masked vectors(logits) to token

In [19]:
import numpy as np

class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V):
        #Q, K, V of size [batch x sequence_length x dim]
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(Q.shape[1])
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context, attn

#sanity checking
q = torch.tensor([[[1.1,1.3],[0.9,0.8]]]).to(device)
k = torch.tensor([[[0.9,1],[0.2,2.1]]]).to(device)
v = torch.tensor([[[1.1,1.3],[0.9,0.8]]]).to(device)
sample = ScaledDotProductAttention().to(device)
sample(q,k,v)

(tensor([[[0.9771, 0.9927],
          [0.9912, 1.0280]]], device='cuda:0'),
 tensor([[[0.3854, 0.6146],
          [0.4559, 0.5441]]], device='cuda:0'))

## Mock shape transformations

In [20]:
# defining sample value matrix, [batch_size x sequence_len x (query_key_dim * n_heads)]
# in this matrix, [0,1,2,3] represents the values for 2 heads across a single word vector
samp_val=torch.tensor([[[0,1,2,3],[4,5,6,7]],[[0,-1,-2,-3],[-4,-5,-6,-7]]])

# dividing into two heads
# [batch_size x sequence_len x query_key_dim x n_heads]
samp_val=samp_val.view(2,2,2,2)

# moving the head dimension next to the batch dimension
# [batch_size x n_heads x sequence_len x query_key_dim]
samp_val = samp_val.permute(0, 3, 1, 2)

# combining batch and head dimension
# [batch_size*n_heads x sequence_len x query_key_dim]
samp_val = samp_val.reshape(-1, 2, 2)

# seperating heads, [batch_size x n_heads x sequence_len x query_key_dim]
samp_val = samp_val.reshape(2,2,2,2)

# moving the head dimension to the end [batch_size x sequence_len x query_key_dim x n_heads]
samp_val = samp_val.permute(0, 2, 3, 1)

#combining the last dim to effectively concatonate the result of the heads
#[batch_size x sequence_len x query_key_dim*n_heads]
samp_val = samp_val.reshape(2, 2, -1)
samp_val

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7]],

        [[ 0, -1, -2, -3],
         [-4, -5, -6, -7]]])

# Implementing MHSA

In [21]:
import torch
import torch.nn as nn

# Define constants
n_heads = 3
query_key_dim = 64
value_dim = 64

class MultiHeadSelfAttention(nn.Module):
    def __init__(self):
        super(MultiHeadSelfAttention, self).__init__()
        # Defining the linear layers that construct the query, key, and value
        self.W_Q = nn.Linear(d_model, query_key_dim * n_heads)   # Projects input to [batch x sequence x (q/k_dim*num_heads)]
        self.W_K = nn.Linear(d_model, query_key_dim * n_heads)   # Projects input to [batch x sequence x (q/k_dim*num_heads)]
        self.W_V = nn.Linear(d_model, value_dim * n_heads)       # Projects input to [batch x sequence x (v_dim*num_heads)]
        self.dot_prod_attn = ScaledDotProductAttention()         # Parameterless system that calculates attention
        self.proj_back = nn.Linear(value_dim * n_heads, d_model) # Projects final output of mhsa back into model dimension

    def forward(self, embedding):

        # passing embedding through dense networks
        qs = self.W_Q(embedding)  # [batch_size x sequence_len x (query_key_dim * n_heads)]
        ks = self.W_K(embedding)  # [batch_size x sequence_len x (query_key_dim * n_heads)]
        vs = self.W_V(embedding)  # [batch_size x sequence_len x (value_dim * n_heads)]

        #dividing out heads
        #[batch_size, sequence_len, q/k/v_dim, n_heads]
        qs = qs.view(batch_size, max_input_length, query_key_dim, n_heads)
        ks = ks.view(batch_size, max_input_length, query_key_dim, n_heads)
        vs = vs.view(batch_size, max_input_length, value_dim, n_heads)

        #moving the head dimension next to the batch dimension
        #[batch_size x n_heads x sequence_len x q/k/v_dim]
        qs = qs.permute(0, 3, 1, 2)
        ks = ks.permute(0, 3, 1, 2)
        vs = vs.permute(0, 3, 1, 2)

        #combining batch and head dimension
        #[batch_size*n_heads x sequence_len x q/k/v_dim]
        qs = qs.reshape(-1, max_input_length, query_key_dim)
        ks = ks.reshape(-1, max_input_length, query_key_dim)
        vs = vs.reshape(-1, max_input_length, value_dim)

        #passing batches/heads of self attention through attn
        #[batch_size*n_heads x sequence_len x q/k/v_dim]
        head_results, _ = self.dot_prod_attn(qs,ks,vs)

        #seperating heads
        #[batch_size x n_heads x sequence_len x v_dim]
        head_results = head_results.reshape(batch_size,n_heads,max_input_length,value_dim)

        #moving the head dimension to the end
        #[batch_size x sequence_len x query_key_dim x n_heads]
        head_results = head_results.permute(0, 2, 3, 1)

        #combining the last dim to effectively concatonate the result of the heads
        #[batch_size x sequence_len x query_key_dim*n_heads]
        head_results = head_results.reshape(batch_size, max_input_length, -1)

        #projecting result of head back into model dimension
        return self.proj_back(head_results)

# Example usage
sample_embeddings = torch.tensor([[[1.1] * d_model] * max_input_length] * batch_size).to(device)
print("Sample embeddings shape:", sample_embeddings.shape)

sample = MultiHeadSelfAttention().to(device)
output = sample(sample_embeddings)
print('Output shape of mhsa:', output.shape)

Sample embeddings shape: torch.Size([128, 64, 256])
Output shape of mhsa: torch.Size([128, 64, 256])


# Point wise feed forward

In [22]:
d_ff = 4*d_model

class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        # (batch_size, len_seq, d_model) -> (batch_size, len_seq, d_ff) -> (batch_size, len_seq, d_model)
        return self.fc2(torch.nn.functional.gelu(self.fc1(x)))

sample = PoswiseFeedForwardNet().to(device)
sample_embeddings = torch.tensor([[[1.1] * d_model] * max_input_length] * batch_size).to(device)
sample(sample_embeddings).shape

torch.Size([128, 64, 256])

# Implementing an encoder block

In [23]:
class EncoderBlock(nn.Module):
    def __init__(self):
        super(EncoderBlock, self).__init__()
        self.mhsa = MultiHeadSelfAttention()
        self.pwff = PoswiseFeedForwardNet()

    def forward(self, x):
        mhsa_output = self.mhsa(x)
        skip1 = mhsa_output + x
        pwff_output = self.pwff(skip1)
        skip2 = skip1+pwff_output
        return skip2

sample = EncoderBlock().to(device)
sample_embeddings = torch.tensor([[[1.1] * d_model] * max_input_length] * batch_size).to(device)
sample(sample_embeddings).shape

torch.Size([128, 64, 256])

# Building BERT

In [24]:
n_layers = 1

class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        #for converting tokens into vector embeddings
        self.embedding = Embedding()
        #encoder blocks
        self.encoder_blocks = nn.ModuleList([EncoderBlock() for _ in range(n_layers)])
        #for decoding a word vector (or tensor of them) into token predictions
        self.decoder = nn.Linear(d_model, tokenizer.vocab_size, bias=False)
        #for converting the first output token into a binary classification
        self.classifier = nn.Linear(d_model, 1, bias=False)

    def forward(self, x, seg, masked_token_locations):

        #x of shape [batch x seq_len x model_dim]
        embeddings = self.embedding(x, seg)
        x = embeddings
        for block in self.encoder_blocks:
            x = block(x)

        #passing first token through classifier
        clsf_logits = self.classifier(x[:,0,:])

        #passing masked tokens through decoder
        masked_token_embeddings = embeddings[masked_token_locations.bool()]
        token_logits = self.decoder(masked_token_embeddings)

        return clsf_logits, token_logits

sample = BERT().to(device)
masked_tokens, masked_token_locations = mask_batch(sequence_tokens_batches[0])
clsf_logits, token_logits = sample(masked_tokens,sentence_location_batches[0], masked_token_locations)
clsf_logits

tensor([[-1.3212],
        [-1.1761],
        [-1.2210],
        [-1.1518],
        [-1.2443],
        [-1.1676],
        [-1.1510],
        [-1.1844],
        [-1.2118],
        [-1.2523],
        [-1.2566],
        [-1.2692],
        [-1.3077],
        [-1.3386],
        [-1.2643],
        [-1.1812],
        [-1.2178],
        [-1.2392],
        [-1.1497],
        [-1.2008],
        [-1.1623],
        [-1.1801],
        [-1.2562],
        [-1.2456],
        [-1.3230],
        [-1.1768],
        [-1.2693],
        [-1.1812],
        [-1.2837],
        [-1.1889],
        [-1.1633],
        [-1.1933],
        [-1.2116],
        [-1.2690],
        [-1.2327],
        [-1.2117],
        [-1.1549],
        [-1.2192],
        [-1.2083],
        [-1.2221],
        [-1.2382],
        [-1.3255],
        [-1.2475],
        [-1.1745],
        [-1.2860],
        [-1.1509],
        [-1.3219],
        [-1.2202],
        [-1.2154],
        [-1.1827],
        [-1.1684],
        [-1.1851],
        [-1.

# Pre-training

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Assume the BERT class and necessary preprocessing functions are already defined

model = BERT().to(device)
token_criterion = nn.CrossEntropyLoss()  # Expect indices, not one-hot vectors
classification_criterion = nn.BCEWithLogitsLoss()  # For logits directly
optimizer = optim.Adam(model.parameters(), lr=0.001)

losses = [[]*1]

#these epochs can take a while, keeping it at a fairly small number
for epoch in range(4):
    for sequence_batch, location_batch, classtarg_batch in tqdm(zip(sequence_tokens_batches, sentence_location_batches, is_positives_batches)):
        # Zeroing out gradients from last iteration
        optimizer.zero_grad()

        # Masking the tokens in the input sequence
        masked_tokens, masked_token_locations = mask_batch(sequence_batch)

        # Generating class and masked token predictions
        clsf_logits, token_logits = model(masked_tokens, location_batch, masked_token_locations)

        # Setting up target for masked token prediction
        masked_token_targets = sequence_batch[masked_token_locations.bool()]

        # Calculating loss for next sentence classification
        loss_clsf = classification_criterion(clsf_logits.squeeze(), classtarg_batch.float())

        # Calculating loss for masked language modeling
        loss_mlm = token_criterion(token_logits, masked_token_targets)

        # Combining losses
        loss = loss_mlm + loss_clsf

        #keeping track of loss across the current epoch
        losses[-1].append(float(loss))

        # Backpropagation
        loss.backward()
        optimizer.step()

    print(f'=======Epoch {epoch} Completed=======')
    print(f'average loss in epoch: {np.mean(losses[-1])}')
    losses.append([])

14it [00:02,  6.91it/s]


=======Epoch 0 Completed=======
average loss in epoch: 10.44417428970337


14it [00:01,  8.09it/s]


=======Epoch 1 Completed=======
average loss in epoch: 8.756936073303223


14it [00:01,  8.04it/s]


=======Epoch 2 Completed=======
average loss in epoch: 8.051488501685006


14it [00:01,  7.94it/s]

=======Epoch 3 Completed=======
average loss in epoch: 7.827998808452061


In [26]:
model

BERT(
  (embedding): Embedding(
    (tok_embed): Embedding(30522, 256)
    (pos_embed): Embedding(64, 256)
    (seg_embed): Embedding(2, 256)
    (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  )
  (encoder_blocks): ModuleList(
    (0): EncoderBlock(
      (mhsa): MultiHeadSelfAttention(
        (W_Q): Linear(in_features=256, out_features=192, bias=True)
        (W_K): Linear(in_features=256, out_features=192, bias=True)
        (W_V): Linear(in_features=256, out_features=192, bias=True)
        (dot_prod_attn): ScaledDotProductAttention()
        (proj_back): Linear(in_features=192, out_features=256, bias=True)
      )
      (pwff): PoswiseFeedForwardNet(
        (fc1): Linear(in_features=256, out_features=1024, bias=True)
        (fc2): Linear(in_features=1024, out_features=256, bias=True)
      )
    )
  )
  (decoder): Linear(in_features=256, out_features=30522, bias=False)
  (classifier): Linear(in_features=256, out_features=1, bias=False)
)

In [27]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

16367936

# Fine tuning on sentiment analysis

In [28]:
fine_tune_ds = load_dataset("fancyzhx/amazon_polarity")

Generating train split:   0%|          | 0/3600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400000 [00:00<?, ? examples/s]

In [29]:
def preprocess_data(data, max_num = 100000):
    data_tokens = []
    data_positional = []
    data_targets = []

    #unpacking data
    for i, elem in enumerate(data):

        #tokenizing the title and content
        sentence1 = elem['title']
        sentence2 = elem['content']
        tokens = tokenizer([sentence1, sentence2])
        sentence1_tokens = tokens['input_ids'][0]
        sentence2_tokens = tokens['input_ids'][1]

        # Trimming down tokens
        if len(sentence1_tokens) + len(sentence2_tokens) > max_input_length:
            sentence1_tokens = [101] + sentence1_tokens[-int(max_input_length / 2) + 1:]
            sentence2_tokens = sentence2_tokens[:int(max_input_length / 2) - 1] + [102]

        # Creating sentence tokens
        sentence_tokens = [0] * len(sentence1_tokens) + [1] * len(sentence2_tokens)

        # Combining and padding
        pad_num = max_input_length - (len(sentence1_tokens) + len(sentence2_tokens))
        sequence_tokens = sentence1_tokens + sentence2_tokens + [0] * pad_num
        sentence_location_tokens = sentence_tokens + [1] * pad_num

        data_tokens.append(sequence_tokens)
        data_positional.append(sentence_location_tokens)
        data_targets.append(elem['label'])

        if i > max_num: break

    return torch.tensor(data_positional), torch.tensor(data_tokens), torch.tensor(data_targets)

#processing data into modeling data
train_pos, train_tok, train_targ = preprocess_data(fine_tune_ds['train'])
test_pos, test_tok, test_targ = preprocess_data(fine_tune_ds['test'])

#moving to training
train_pos = train_pos.to(device)
train_tok = train_tok.to(device)
train_targ = train_targ.to(device)

#moving to testing
test_pos = test_pos.to(device)
test_tok = test_tok.to(device)
test_targ = test_targ.to(device)

print(train_tok)

tensor([[  101,   101, 24646,  ...,     0,     0,     0],
        [  101,   101,  1996,  ...,     0,     0,     0],
        [  101,   101,  6429,  ...,     0,     0,     0],
        ...,
        [  101,   101,  2438,  ...,     0,     0,     0],
        [  101,   101,  2307,  ...,     0,     0,     0],
        [  101,   101,  9458,  ...,     0,     0,     0]], device='cuda:0')


In [30]:
# Replacing classification head with a new head
# the new training objective is still binary classification,
# except these parameters will be used to decide if a
# review was positive or negative
model.classifier = nn.Linear(d_model, 1, bias=False).to(device)

# resetting the optimizer to have access to the parameters of the new head
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [31]:
ft_losses = [[]*1]
ft_test_acc = []

for epoch in range(5):
    for i in tqdm(range(0, train_pos.shape[0], batch_size)):

        if i+batch_size>=train_pos.shape[0]:
            break

        #getting batch
        train_pos_batch = train_pos[i:i+batch_size]
        train_tok_batch = train_tok[i:i+batch_size]
        train_targ_batch = train_targ[i:i+batch_size]

        # Zeroing out gradients from last iteration
        optimizer.zero_grad()

        # Masking the tokens in the input sequence
        masked_tokens, masked_token_locations = mask_batch(train_tok_batch)

        # Generating class and masked token predictions
        clsf_logits, token_logits = model(masked_tokens, train_pos_batch, masked_token_locations)

        # Setting up target for masked token prediction
        masked_token_targets = sequence_batch[masked_token_locations.bool()]

        # Calculating loss for next sentence classification
        loss_clsf = classification_criterion(clsf_logits.squeeze(), train_targ_batch.float())

        # Calculating loss for masked language modeling
        loss_mlm = token_criterion(token_logits, masked_token_targets)

        # Combining losses
        loss = loss_mlm + loss_clsf

        ft_losses[-1].append(float(loss))

        # Backpropagation
        loss.backward()
        optimizer.step()

    print(f'=======Epoch {epoch} Completed=======')
    print(f'average loss in epoch: {np.mean(ft_losses[-1])}')
    losses.append([])

100%|█████████▉| 781/782 [01:21<00:00,  9.62it/s]


=======Epoch 0 Completed=======
average loss in epoch: 5.109838737263112


100%|█████████▉| 781/782 [01:20<00:00,  9.69it/s]


=======Epoch 1 Completed=======
average loss in epoch: 4.900701988674462


100%|█████████▉| 781/782 [01:20<00:00,  9.66it/s]


=======Epoch 2 Completed=======
average loss in epoch: 4.81253920830675


100%|█████████▉| 781/782 [01:20<00:00,  9.68it/s]


=======Epoch 3 Completed=======
average loss in epoch: 4.760771188784806


100%|█████████▉| 781/782 [01:20<00:00,  9.65it/s]

=======Epoch 4 Completed=======
average loss in epoch: 4.723997773892138


In [32]:
is_correct = []
predicted_class = []
original_class = []

for i in tqdm(range(0, test_pos.shape[0], batch_size)):

    if i+batch_size>=test_pos.shape[0]:
            break

    #getting batch
    test_pos_batch = test_pos[i:i+batch_size]
    test_tok_batch = test_tok[i:i+batch_size]
    test_targ_batch = test_targ[i:i+batch_size]

    #making prediction, not masking anything
    clsf_logits, _ = model(test_tok_batch, test_pos_batch, torch.zeros(test_pos_batch.shape))

    #converting logits to probabilities then rounding to classifications
    res = torch.sigmoid(clsf_logits).round().squeeze()

    #keeping track of the original class (positive or negative) and if the model was correct
    original_class.extend(np.array(test_targ_batch.to('cpu')))
    is_correct.extend(np.array((res == test_targ_batch).to('cpu')))
    predicted_class.extend(np.array(res.detach().to('cpu')))

100%|█████████▉| 781/782 [00:02<00:00, 304.01it/s]


In [33]:
#accuracy
sum(list(is_correct))/len(is_correct)

0.8422795294494239

In [34]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(original_class, predicted_class))

              precision    recall  f1-score   support

           0       0.88      0.79      0.83     49405
           1       0.81      0.90      0.85     50563

    accuracy                           0.84     99968
   macro avg       0.85      0.84      0.84     99968
weighted avg       0.85      0.84      0.84     99968



# Acknowledgements

* https://medium.com/towards-data-science/bert-intuitively-and-exhaustively-explained-48a24ecc1c8a
* https://github.com/DanielWarfield1/MLWritingAndResearch/blob/main/BERTFromScratch.ipynb